In [6]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import os
import wget
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [7]:
#code by pythonjar, not me
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [8]:
headers={'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36'}

In [9]:
# from selenium import webdriver

# Specify the path to chromedriver.exe (download and save it on your computer)
chrome_driver_path = 'C:/Users/hp/chromedriver.exe'

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = "C:/Program Files/Google/Chrome/Application/chrome.exe"  # Update with the actual path to your Chrome executable

# Create the WebDriver with the specified path and options
driver = webdriver.Chrome(options=chrome_options)

# Open the webpage
driver.get("https://www.amazon.in/")
# driver.get("https://www.flipkart.com/")

In [10]:
# a
css_selector = "input[name='field-keywords']"

input_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))
input_field.clear()
input_field.send_keys("lego")

search_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='submit']")))

search_button.click()


In [ ]:
search = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//span[@class='a-size-base-plus a-color-base a-text-normal'])[1]")))

search.click()

In [14]:
# link_element = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a")

# # Extract the link
# link = link_element.get_attribute("href")

# # Print the link
# print(link)

# # Assign the extracted link to the URL variable
# URL = link

# # Define headers
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
#     "Accept-Encoding": "gzip, deflate",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#     "DNT": "1",
#     "Connection": "close",
#     "Upgrade-Insecure-Requests": "1"
# }

# # Fetch the webpage
# page = requests.get(URL, headers=headers)

# # Parse the webpage content
# soup1 = BeautifulSoup(page.content, "html.parser")

# # Find the title element and extract the text
# title = soup1.find(id='productTitle').get_text()
# table= soup1.find(id='productDetails_techSpec_section_1').get_text()


# # Print the extracted title
# print(table)
# print(title)


In [11]:
import pandas as pd

# Initialize an empty DataFrame to store the technical specifications
tech_spec_df_list = []

# Initialize an empty DataFrame to store the title and price information
title_price_df_list = []

while True:
    for i in range(1, 74):
        if i == 58:  # Skip iteration if i is equal to 58
            continue
        try:
            # Extract the URL from the link element
            # Adjust the XPath expression to target links that are not ads
#             link_xpath = f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal' and not(ancestor::div[contains(@class, 'ad')])])[{i}]"
#             link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, link_xpath)))

            link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{i}]")))
            url = link_element.get_attribute("href")
            # Check if the href contains the substring to skip
            if "gp/video/ssoredirect" in url:
                continue  # Skip to the next iteration if href contains the substring to skip

            # Open the URL in a new tab
            driver.execute_script(f"window.open('{url}', '_blank')")

            # Switch to the newly opened tab
            driver.switch_to.window(driver.window_handles[-1])

            # Wait for the title and price elements to be present
            try:
                title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
                fi_title = title_element.text.strip()
            except:
                fi_title = ""  # Skip to the next iteration if title element not found

            try:
                price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
                fi_price = price_element.text.strip()
            except:
                fi_price=""  # Skip to the next iteration if price element not found

            # Append the title and price information to the DataFrame list
            title_price_df_list.append({'Title': fi_title, 'Price': fi_price})

        except:
            continue  # Skip to the next iteration if link element not found

        try:
            # Wait for the technical specifications section to be present
            tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

            # Extract the outer HTML of the technical specifications section
            tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

            # Parse the HTML content
            soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

            # Find all rows in the table
            rows = soup.find('table').find_all('tr')

            # Extract column headers and data
            headers = []
            data = []
            for row in rows:
                cols = row.find_all(['th', 'td'])
                if len(cols) == 2:  # Ensure there are two columns
                    headers.append(cols[0].get_text(strip=True))
                    data.append(cols[1].get_text(strip=True))

            # Pad data with empty strings if it's shorter than headers
            data += [''] * (len(headers) - len(data))

            # Create a DataFrame for technical specifications
            tech_spec_df = pd.DataFrame([data], columns=headers)

            # Append the technical specifications DataFrame to the list
            tech_spec_df_list.append(tech_spec_df)

        except:
            pass

        # Close the current tab and switch back to the previous tab
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass

    # Check if there is a "Next" button
    try:
        search = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//a[normalize-space()='Next'])[1]")))
        search.click()
    except:
        break

# Combine title and price information into a DataFrame
title_price_df = pd.DataFrame(title_price_df_list)

# Combine technical specifications DataFrames into a single DataFrame
tech_spec_combined_df = pd.concat(tech_spec_df_list, ignore_index=True)

# Print DataFrames
print("Title and Price DataFrame:")
print(title_price_df)

print("\nTechnical Specifications DataFrame:")
print(tech_spec_combined_df)

# # Merge the DataFrames
# merged_df = title_price_df.join(tech_spec_combined_df)

# # Print the merged DataFrame
# print("\nMerged DataFrame:")
# print(merged_df)


Title and Price DataFrame:
                                                 Title  \
0    LEGO Icons PAC-Man Arcade 10323 Building Kit (...   
1    LEGO Ideas Disney Hocus Pocus: The Sanderson S...   
2    LEGO Icons Eldorado Fortress 10320 Building Ki...   
3    LEGO Ideas Viking Village 21343 Building Set f...   
4    LEGO Icons PAC-Man Arcade 10323 Building Kit (...   
..                                                 ...   
391  LEGO City Custom Car Garage 60389 Building Toy...   
392  ICW Multificence Changeable Series Spinosaurus...   
393  LEGO Star Wars The Mandalorian Helmet 75328 Bu...   
394  Kawaii Kart | Architect Brick Toys Sunset Race...   
395  LEGO Architecture L&Marks Collection Taj Mahal...   

                                 Price  
0      -35%\n₹19,499\nM.R.P.:\n₹29,999  
1      -35%\n₹10,399\nM.R.P.:\n₹15,999  
2      -40%\n₹14,399\nM.R.P.:\n₹23,999  
3       -35%\n₹9,749\nM.R.P.:\n₹14,999  
4      -35%\n₹19,499\nM.R.P.:\n₹29,999  
..                            

In [14]:
# Merge the DataFrames
merged_df = title_price_df.join(tech_spec_combined_df)

# Print the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df)


Merged DataFrame:
                                                 Title  \
0    LEGO Icons PAC-Man Arcade 10323 Building Kit (...   
1    LEGO Ideas Disney Hocus Pocus: The Sanderson S...   
2    LEGO Icons Eldorado Fortress 10320 Building Ki...   
3    LEGO Ideas Viking Village 21343 Building Set f...   
4    LEGO Icons PAC-Man Arcade 10323 Building Kit (...   
..                                                 ...   
391  LEGO City Custom Car Garage 60389 Building Toy...   
392  ICW Multificence Changeable Series Spinosaurus...   
393  LEGO Star Wars The Mandalorian Helmet 75328 Bu...   
394  Kawaii Kart | Architect Brick Toys Sunset Race...   
395  LEGO Architecture L&Marks Collection Taj Mahal...   

                                 Price Language: Model Number  \
0      -35%\n₹19,499\nM.R.P.:\n₹29,999  ‎English       ‎10323   
1      -35%\n₹10,399\nM.R.P.:\n₹15,999  ‎English       ‎21341   
2      -40%\n₹14,399\nM.R.P.:\n₹23,999       NaN     ‎6426509   
3       -35%\n₹9,749\nM.

In [16]:
merged_df.to_excel("amazon_data.xlsx", index=False)

print("Data has been saved to merged_data.xlsx file.")

Data has been saved to merged_data.xlsx file.


In [6]:
import pandas as pd

# Initialize an empty DataFrame to store the technical specifications
tech_spec_df_list = []

# Initialize an empty DataFrame to store the title and price information
title_price_df_list = []

while True:
    for i in range(1, 74):
        try:
            # Extract the URL from the link element
            link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{i}]")))
            url = link_element.get_attribute("href")

            # Open the URL in a new tab
            driver.execute_script(f"window.open('{url}', '_blank')")

            # Switch to the newly opened tab
            driver.switch_to.window(driver.window_handles[-1])

            # Wait for the title and price elements to be present
            title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
            fi_title = title_element.text.strip()

            price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
            fi_price = price_element.text.strip()

            # Append the title and price information to the DataFrame list
            title_price_df_list.append({'Title': fi_title, 'Price': fi_price})

        except:
            pass

        try:
            # Wait for the technical specifications section to be present
            tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

            # Extract the outer HTML of the technical specifications section
            tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

            # Parse the HTML content
            soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

            # Find all rows in the table
            rows = soup.find('table').find_all('tr')

            # Extract column headers and data
            headers = []
            data = []
            for row in rows:
                cols = row.find_all(['th', 'td'])
                if len(cols) == 2:  # Ensure there are two columns
                    headers.append(cols[0].get_text(strip=True))
                    data.append(cols[1].get_text(strip=True))

            # Pad data with empty strings if it's shorter than headers
            data += [''] * (len(headers) - len(data))

            # Create a DataFrame for technical specifications
            tech_spec_df = pd.DataFrame([data], columns=headers)

            # Append the technical specifications DataFrame to the list
            tech_spec_df_list.append(tech_spec_df)

        except:
            pass

        # Close the current tab and switch back to the previous tab
        try:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass

    # Check if there is a "Next" button
    try:
        search = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//a[normalize-space()='Next'])[1]")))
        search.click()
    except:
        break

# Combine title and price information into a DataFrame
title_price_df = pd.DataFrame(title_price_df_list)

# Combine technical specifications DataFrames into a single DataFrame
tech_spec_combined_df = pd.concat(tech_spec_df_list, ignore_index=True)

# Print DataFrames
print("Title and Price DataFrame:")
print(title_price_df)

print("\nTechnical Specifications DataFrame:")
print(tech_spec_combined_df)

# Merge the DataFrames
merged_df = title_price_df.join(tech_spec_combined_df)

# Print the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df)


Title and Price DataFrame:
                                               Title  \
0  LEGO Technic 2022 Ford Gt 42154 Building Kit f...   
1  LEGO Icons PAC-Man Arcade 10323 Building Kit (...   
2  LEGO Technic Kawasaki Ninja H2R Motorcycle Set...   
3  LEGO Ideas Disney Hocus Pocus: The Sanderson S...   
4  LEGO Technic Peugeot 9X8 24H Le Mans Hybrid Hy...   
5  LEGO Icons PAC-Man Arcade 10323 Building Kit (...   
6  LEGO Creator 3in1 Cyber Drone 31111 Building K...   
7  LEGO Technic Monster Jam Monster Mutt Dalmatia...   
8  LEGO Creator 3in1 Off-Road Buggy 31123 Buildin...   
9  LEGO Technic Peugeot 9X8 24H Le Mans Hybrid Hy...   

                             Price  
0   -40%\n₹8,999\nM.R.P.:\n₹14,999  
1  -35%\n₹19,499\nM.R.P.:\n₹29,999  
2                           ₹9,999  
3  -35%\n₹10,399\nM.R.P.:\n₹15,999  
4  -50%\n₹10,999\nM.R.P.:\n₹21,999  
5  -35%\n₹19,499\nM.R.P.:\n₹29,999  
6      -25%\n₹899\nM.R.P.:\n₹1,199  
7    -27%\n₹2,199\nM.R.P.:\n₹2,999  
8      -50%\n₹949\nM.R.

In [7]:
merged_df

,Title,Price,Educational Objective(s),Language:,Model Number,Number of Puzzle Pieces,Assembly Required,Batteries Required,Batteries Included,Material Type(s),...,Colour,Product Dimensions,Batteries,Item model number,Manufacturer recommended age,Manufacturer,Country of Origin,Item Weight,Manufacturer's Suggested Maximum Weight,Radio Control Number of Bands
0,LEGO Technic 2022 Ford Gt 42154 Building Kit f...,"-40%\n₹8,999\nM.R.P.:\n₹14,999",‎LEGO,‎English,‎42154,‎1466,‎Yes,‎No,‎No,‎Plastic,...,‎Multicolor,‎48 x 28.2 x 9.1 cm; 1 g,‎1 Unknown batteries required.,‎42154,‎4 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎1 g,NaN,NaN
1,LEGO Icons PAC-Man Arcade 10323 Building Kit (...,"-35%\n₹19,499\nM.R.P.:\n₹29,999",NaN,‎English,‎10323,‎2651,‎No,‎No,‎Yes,‎Plastic,...,‎Black,‎12.4 x 58.2 x 48 cm; 3.64 kg,‎2 LR41 batteries required. (included),‎10323,‎18 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎3 kg 640 g,NaN,NaN
2,LEGO Technic Kawasaki Ninja H2R Motorcycle Set...,"₹9,999",NaN,NaN,‎42170,‎643 pièce(s),‎Yes,‎No,‎No,‎Plastic,...,‎Multicolor,‎38.2 x 26.2 x 7.05 cm; 851 g,NaN,‎42170,‎10 years and up,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎851 g,NaN,NaN
3,LEGO Ideas Disney Hocus Pocus: The Sanderson S...,"-35%\n₹10,399\nM.R.P.:\n₹15,999",‎Creative Thinking,‎English,‎21341,‎2316,‎No,‎Yes,‎Yes,‎Plastic,...,‎Multicolor,‎58.2 x 37.8 x 10.3 cm; 2.35 kg,NaN,‎21341,‎18 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎2 kg 350 g,NaN,NaN
4,LEGO Technic Peugeot 9X8 24H Le Mans Hybrid Hy...,"-50%\n₹10,999\nM.R.P.:\n₹21,999",‎Creative Thinking,‎English,‎42156,‎1775,‎Yes,‎No,‎No,‎Plastic,...,‎Multicolor,‎48 x 37.8 x 11.2 cm; 1 g,NaN,‎42156,‎18 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎1 g,NaN,NaN
5,LEGO Icons PAC-Man Arcade 10323 Building Kit (...,"-35%\n₹19,499\nM.R.P.:\n₹29,999",NaN,‎English,‎10323,‎2651,‎No,‎No,‎Yes,‎Plastic,...,‎Black,‎12.4 x 58.2 x 48 cm; 3.64 kg,‎2 LR41 batteries required. (included),‎10323,‎18 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎3 kg 640 g,NaN,NaN
6,LEGO Creator 3in1 Cyber Drone 31111 Building K...,"-25%\n₹899\nM.R.P.:\n₹1,199","‎Creative Thinking, Construction Skills, Role ...","‎English, French, Danish, German, Spanish",‎31111,‎113,‎Yes,‎No,‎No,‎Plastic,...,‎Multicolor,‎4.6 x 19.1 x 14.1 cm; 150 g,‎1 Unknown batteries required.,‎31111,‎6 years and up,‎LEGO,‎Czech Republic,‎150 g,‎20 Milligrams,‎1
7,LEGO Technic Monster Jam Monster Mutt Dalmatia...,"-27%\n₹2,199\nM.R.P.:\n₹2,999",‎Imagination Development,‎English,‎42150,‎244,‎Yes,‎No,‎No,‎Plastic,...,‎Multicolor,‎26.2 x 14.1 x 9.1 cm; 410 g,‎1 Unknown batteries required.,‎42150,‎7 years and up,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎410 g,NaN,NaN
8,LEGO Creator 3in1 Off-Road Buggy 31123 Buildin...,"-50%\n₹949\nM.R.P.:\n₹1,899","‎Creative Thinking, Role Play",NaN,‎31123,‎160,‎Yes,‎No,NaN,‎Plastic,...,‎Multicolor,‎26.2 x 14.1 x 6.1 cm; 280 g,NaN,‎31123,‎7 years and up,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎280 g,NaN,NaN
9,LEGO Technic Peugeot 9X8 24H Le Mans Hybrid Hy...,"-50%\n₹10,999\nM.R.P.:\n₹21,999",‎Creative Thinking,‎English,‎42156,‎1775,‎Yes,‎No,‎No,‎Plastic,...,‎Multicolor,‎48 x 37.8 x 11.2 cm; 1 g,NaN,‎42156,‎18 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎1 g,NaN,NaN


In [36]:
search = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//a[normalize-space()='Next'])[1]")))

search.click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007FF76F5F0EC4]
	(No symbol) [0x00007FF76F5F1052]
	(No symbol) [0x00007FF76F5E18A4]
	BaseThreadInitThunk [0x00007FFB4F017344+20]
	RtlUserThreadStart [0x00007FFB4F2E26B1+33]


In [37]:
# Wait for the link element to be clickable
time.sleep(20)
link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{}]")))

# Extract the link
link = link_element.get_attribute("href")

# Print the link
print(link)

# Assign the extracted link to the URL variable
URL = link

# Fetch the webpage
driver.get(URL)

# Wait for the title and price elements to be present
title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))

# Extract the title and price
fi_title = title_element.text.strip()
fi_price = price_element.text.strip()

# Wait for the technical specifications section to be present
tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

# Extract the outer HTML of the technical specifications section
tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

# Parse the HTML content
soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

# Find all rows in the table
rows = soup.find('table').find_all('tr')

# Extract column headers and data
headers = []
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    if len(cols) == 2:  # Ensure there are two columns
        headers.append(cols[0].get_text(strip=True))
        data.append(cols[1].get_text(strip=True))

# Add title and price at the beginning of the data list
data.insert(0, fi_price)
data.insert(0, fi_title)

# Create DataFrame
df = pd.DataFrame([data], columns=['Title', 'Price'] + headers)

# Print DataFrame
print(df)
df

KeyboardInterrupt: 

In [16]:
/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[68]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a
/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[68]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a
/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[67]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a
/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[11]/div/div/div/div/div/div/div/span/div/div[1]/div/div/div/ol/li[3]/div/div/div/span/div/div/div[2]/div[1]/h2/a
/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[69]/div/div/span/div/div/div/div/div[2]/div/span/div/div/div/div[3]/div/div/div[1]/h2/a
/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[11]/div/div/div/div/div/div/div/span/div/div[1]/div/div/div/ol/li[3]/div/div/div/span/div/div/div[2]/div[1]/h2

import pandas as pd

# Initialize an empty DataFrame to store the technical specifications
tech_spec_df_list = []

# Initialize an empty DataFrame to store the title and price information
title_price_df_list = []

# Iterate through XPATH to extract URLs and fetch information
for i in range(11, 74):
    try:
        # Extract the URL from the link element
        link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{i}]")))
        url = link_element.get_attribute("href")
        
        # Open the URL in a new tab
        driver.execute_script(f"window.open('{url}', '_blank')")
        
        # Switch to the newly opened tab
        driver.switch_to.window(driver.window_handles[-1])
        
        # Wait for the title and price elements to be present
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        fi_title = title_element.text.strip()
        
        price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
        fi_price = price_element.text.strip()
        
        # Append the title and price information to the DataFrame list
        title_price_df_list.append({'Title': fi_title, 'Price': fi_price})
        
    except:
        pass
    
    try:
        # Wait for the technical specifications section to be present
        tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))
        
        # Extract the outer HTML of the technical specifications section
        tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")
        
        # Parse the HTML content
        soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')
        
        # Find all rows in the table
        rows = soup.find('table').find_all('tr')
        
        # Extract column headers and data
        headers = []
        data = []
        for row in rows:
            cols = row.find_all(['th', 'td'])
            if len(cols) == 2:  # Ensure there are two columns
                headers.append(cols[0].get_text(strip=True))
                data.append(cols[1].get_text(strip=True))
        
        # Pad data with empty strings if it's shorter than headers
        data += [''] * (len(headers) - len(data))
        
        # Create a DataFrame for technical specifications
        tech_spec_df = pd.DataFrame([data], columns=headers)
        
        # Append the technical specifications DataFrame to the list
        tech_spec_df_list.append(tech_spec_df)
        
    except:
        pass
    
    # Close the current tab and switch back to the previous tab
    try:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except:
        pass

# Combine title and price information into a DataFrame
title_price_df = pd.DataFrame(title_price_df_list)

# Combine technical specifications DataFrames into a single DataFrame
tech_spec_combined_df = pd.concat(tech_spec_df_list, ignore_index=True)

# Print DataFrames
print("Title and Price DataFrame:")
print(title_price_df)

print("\nTechnical Specifications DataFrame:")
print(tech_spec_combined_df)

# Close the browser
# driver.quit()
title_price_df
tech_spec_combined_df
merged_df = title_price_df.join(tech_spec_combined_df)
merged_df

ValueError: No objects to concatenate

In [29]:
title_price_df

,Title,Price
0,LEGO Ideas Disney Hocus Pocus: The Sanderson S...,"-35%\n₹10,399\nM.R.P.:\n₹15,999"
1,LEGO Creator 3in1 Downtown Noodle Shop 31131 B...,"-35%\n₹3,574\nM.R.P.:\n₹5,499"
2,LEGO Harry Potter Hogwarts Magical Trunk 76399...,"-7%\n₹7,899\nM.R.P.:\n₹8,499"
3,LEGO Creator 3in1 Off-Road Buggy 31123 Buildin...,"-50%\n₹949\nM.R.P.:\n₹1,899"
4,Lego Creator Vintage Motorcycle 31135 Building...,"-15%\n₹1,614\nM.R.P.:\n₹1,899"
5,"LEGO Bonsai Tree 10281 Building Kit (878 Pcs),...","-35%\n₹3,899\nM.R.P.:\n₹5,999"
6,LEGO DC Batman Construction Figure 76259 Build...,"-25%\n₹2,624\nM.R.P.:\n₹3,499"
7,LEGO Speed Champions Ferrari 812 Competizione ...,"-4%\n₹2,874\nM.R.P.:\n₹2,999"
8,LEGO Creator Exotic Parrot 31136 Building Toy ...,"-12%\n₹2,649\nM.R.P.:\n₹2,999"
9,LEGO City Police Bike Car Chase 60392 Building...,"-10%\n₹1,079\nM.R.P.:\n₹1,199"


In [30]:
tech_spec_combined_df

,Educational Objective(s),Language:,Model Number,Number of Puzzle Pieces,Assembly Required,Batteries Required,Batteries Included,Material Type(s),Colour,Product Dimensions,...,Manufacturer,Country of Origin,Item Weight,Remote Control Included?,Batteries,Manufacturer's Suggested Maximum Weight,Remote Control Type,Scale,Material Composition,Imported By
0,‎Creative Thinking,‎English,‎21341,‎2316,‎No,‎Yes,‎Yes,‎Plastic,‎Multicolor,‎58.2 x 37.8 x 10.3 cm; 2.35 kg,...,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎2 kg 350 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,‎Analytical Skill,NaN,‎31131,‎569,‎Yes,‎No,NaN,‎Plastic,‎Multicolor,‎38.2 x 26.2 x 5.65 cm; 880 g,...,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎880 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,‎Creative Thinking,NaN,‎76399,‎603,‎Yes,‎No,NaN,‎Plastic,‎Multicolor,‎38.2 x 26.2 x 5.65 cm; 760 g,...,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎760 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"‎Creative Thinking, Role Play",NaN,‎31123,‎160,‎Yes,‎No,NaN,‎Plastic,‎Multicolor,‎26.2 x 14.1 x 6.1 cm; 280 g,...,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎280 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,‎31135,‎128,‎Yes,‎No,‎No,‎Plastic,‎Multicolor,‎19.1 x 14.1 x 4.6 cm; 150 g,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",NaN,‎150 g,‎No,‎1 Unknown batteries required.,NaN,NaN,NaN,NaN,NaN
5,"‎Creative Thinking, Imagination Development","‎English, French, Danish, German",‎6332925,‎878,‎Yes,‎No,‎No,‎Plastic,‎Multicolor,‎7.05 x 26.2 x 38.2 cm; 730 g,...,‎LEGO,‎China,‎730 g,‎No,‎1 Unknown batteries required.,‎20 Milligrams,NaN,NaN,NaN,NaN
6,NaN,‎French,‎76259,‎275,‎Yes,‎No,‎No,‎Plastic,‎Multicolor,‎19.1 x 26.2 x 6.1 cm; 340 g,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎340 g,‎No,NaN,NaN,NaN,NaN,NaN,NaN
7,‎Role Play,‎English,‎76914,‎261,‎Yes,‎No,‎No,‎Plastic,‎Multicolor,‎26.2 x 14.1 x 7.2 cm; 323 g,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎323 g,‎No,‎1 Unknown batteries required.,NaN,NaN,NaN,NaN,NaN
8,‎Creative Thinking,‎English,‎31136,‎253,‎Yes,‎No,‎No,‎Plastic,‎Multicolor,‎19.1 x 26.2 x 6.1 cm; 350 g,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎350 g,‎No,‎1 Unknown batteries required.,NaN,NaN,NaN,NaN,NaN
9,‎Role Play,‎English,‎60392,‎59,‎Yes,‎No,‎No,‎Plastic,‎Multicolor,‎15.7 x 14.1 x 4.5 cm; 110 g,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎110 g,‎No,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
merged_df = title_price_df.join(tech_spec_combined_df)

# Print the merged DataFrame
print(merged_df)
merged_df

                                                Title  \
0   LEGO Ideas Disney Hocus Pocus: The Sanderson S...   
1   LEGO Creator 3in1 Downtown Noodle Shop 31131 B...   
2   LEGO Harry Potter Hogwarts Magical Trunk 76399...   
3   LEGO Creator 3in1 Off-Road Buggy 31123 Buildin...   
4   Lego Creator Vintage Motorcycle 31135 Building...   
5   LEGO Bonsai Tree 10281 Building Kit (878 Pcs),...   
6   LEGO DC Batman Construction Figure 76259 Build...   
7   LEGO Speed Champions Ferrari 812 Competizione ...   
8   LEGO Creator Exotic Parrot 31136 Building Toy ...   
9   LEGO City Police Bike Car Chase 60392 Building...   
10  LEGO Friends Mobile Fashion Boutique Building ...   
11  LEGO Minecraft The Creeper Ambush 21177 Buildi...   
12  LEGO City Cement Mixer Truck 60325 Building Ki...   
13  LEGO Speed Champions 2023 McLaren Formula 1 Ra...   
14  LEGO Creator Magical Unicorn 31140 Building To...   
15  LEGO Animal Free Builds-Make It Yours,Multicol...   
16  LEGO Ninjago Destiny'S Boun

,Title,Price,Educational Objective(s),Language:,Model Number,Number of Puzzle Pieces,Assembly Required,Batteries Required,Batteries Included,Material Type(s),...,Manufacturer,Country of Origin,Item Weight,Remote Control Included?,Batteries,Manufacturer's Suggested Maximum Weight,Remote Control Type,Scale,Material Composition,Imported By
0,LEGO Ideas Disney Hocus Pocus: The Sanderson S...,"-35%\n₹10,399\nM.R.P.:\n₹15,999",‎Creative Thinking,‎English,‎21341,‎2316,‎No,‎Yes,‎Yes,‎Plastic,...,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎2 kg 350 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEGO Creator 3in1 Downtown Noodle Shop 31131 B...,"-35%\n₹3,574\nM.R.P.:\n₹5,499",‎Analytical Skill,NaN,‎31131,‎569,‎Yes,‎No,NaN,‎Plastic,...,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎880 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LEGO Harry Potter Hogwarts Magical Trunk 76399...,"-7%\n₹7,899\nM.R.P.:\n₹8,499",‎Creative Thinking,NaN,‎76399,‎603,‎Yes,‎No,NaN,‎Plastic,...,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎760 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LEGO Creator 3in1 Off-Road Buggy 31123 Buildin...,"-50%\n₹949\nM.R.P.:\n₹1,899","‎Creative Thinking, Role Play",NaN,‎31123,‎160,‎Yes,‎No,NaN,‎Plastic,...,"‎LEGO, LEG GODT India Private Limited",‎Czech Republic,‎280 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lego Creator Vintage Motorcycle 31135 Building...,"-15%\n₹1,614\nM.R.P.:\n₹1,899",NaN,NaN,‎31135,‎128,‎Yes,‎No,‎No,‎Plastic,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",NaN,‎150 g,‎No,‎1 Unknown batteries required.,NaN,NaN,NaN,NaN,NaN
5,"LEGO Bonsai Tree 10281 Building Kit (878 Pcs),...","-35%\n₹3,899\nM.R.P.:\n₹5,999","‎Creative Thinking, Imagination Development","‎English, French, Danish, German",‎6332925,‎878,‎Yes,‎No,‎No,‎Plastic,...,‎LEGO,‎China,‎730 g,‎No,‎1 Unknown batteries required.,‎20 Milligrams,NaN,NaN,NaN,NaN
6,LEGO DC Batman Construction Figure 76259 Build...,"-25%\n₹2,624\nM.R.P.:\n₹3,499",NaN,‎French,‎76259,‎275,‎Yes,‎No,‎No,‎Plastic,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎340 g,‎No,NaN,NaN,NaN,NaN,NaN,NaN
7,LEGO Speed Champions Ferrari 812 Competizione ...,"-4%\n₹2,874\nM.R.P.:\n₹2,999",‎Role Play,‎English,‎76914,‎261,‎Yes,‎No,‎No,‎Plastic,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎323 g,‎No,‎1 Unknown batteries required.,NaN,NaN,NaN,NaN,NaN
8,LEGO Creator Exotic Parrot 31136 Building Toy ...,"-12%\n₹2,649\nM.R.P.:\n₹2,999",‎Creative Thinking,‎English,‎31136,‎253,‎Yes,‎No,‎No,‎Plastic,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎350 g,‎No,‎1 Unknown batteries required.,NaN,NaN,NaN,NaN,NaN
9,LEGO City Police Bike Car Chase 60392 Building...,"-10%\n₹1,079\nM.R.P.:\n₹1,199",‎Role Play,‎English,‎60392,‎59,‎Yes,‎No,‎No,‎Plastic,...,"‎LEGO Production s.r.o, Jutska, 2779 Psc-27201...",‎Czech Republic,‎110 g,‎No,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Initialize an empty DataFrame to store the results
df_list = []

# Iterate through XPATH to extract URLs and fetch information
for i in range(1, 74):
    try:
        # Extract the URL from the link element
        link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{i}]")))
        url = link_element.get_attribute("href")
        
        # Open the URL in a new tab
        driver.execute_script(f"window.open('{url}', '_blank')")
        
        # Switch to the newly opened tab
        driver.switch_to.window(driver.window_handles[-1])
        
        # Wait for the title and price elements to be present
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        fi_title = title_element.text.strip()
    except:
        fi_title = ""
    
    try:
        price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
        fi_price = price_element.text.strip()
    except:
        fi_price = ""
    
    try:
        # Wait for the technical specifications section to be present
        tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))
        
        # Extract the outer HTML of the technical specifications section
        tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")
        
        # Parse the HTML content
        soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')
        
        # Find all rows in the table
        rows = soup.find('table').find_all('tr')
        
        # Extract column headers and data
        headers = []
        data = []
        for row in rows:
            cols = row.find_all(['th', 'td'])
            if len(cols) == 2:  # Ensure there are two columns
                headers.append(cols[0].get_text(strip=True))
                data.append(cols[1].get_text(strip=True))
        
        # Add title and price at the beginning of the data list
        data.insert(0, fi_price)
        data.insert(0, fi_title)
        
        # Pad data with empty strings if it's shorter than headers
        data += [''] * (len(headers) - len(data))
    except:
        headers = []
        data = [''] * len(headers)
    
    # Append the extracted information to the DataFrame list
    df_list.append(data)
    
    # Close the current tab and switch back to the previous tab
    try:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except:
        pass

# Create DataFrame
df = pd.DataFrame(df_list, columns=['Title', 'Price'] + headers)

# Print DataFrame
print(df)

# Close the browser
# driver.quit()
df

ValueError: 2 columns passed, passed data had 20 columns

In [9]:
# Initialize an empty DataFrame to store the results
df_list = []

# Iterate through XPATH to extract URLs and fetch information
for i in range(1, 74):
    try:
        # Extract the URL from the link element
        link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{i}]")))
        url = link_element.get_attribute("href")
        
        # Open the URL in a new tab
        driver.execute_script(f"window.open('{url}', '_blank')")
        
        # Switch to the newly opened tab
        driver.switch_to.window(driver.window_handles[-1])
        
        # Wait for the title and price elements to be present
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        fi_title = title_element.text.strip()
    except:
        fi_title = ""
    
    try:
        price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
        fi_price = price_element.text.strip()
    except:
        fi_price = ""
    
    try:
        # Wait for the technical specifications section to be present
        tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))
        
        # Extract the outer HTML of the technical specifications section
        tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")
        
        # Parse the HTML content
        soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')
        
        # Find all rows in the table
        rows = soup.find('table').find_all('tr')
        
        # Extract column headers and data
        headers = []
        data = []
        for row in rows:
            cols = row.find_all(['th', 'td'])
            if len(cols) == 2:  # Ensure there are two columns
                headers.append(cols[0].get_text(strip=True))
                data.append(cols[1].get_text(strip=True))
        
        # Add title and price at the beginning of the data list
        data.insert(0, fi_price)
        data.insert(0, fi_title)
    except:
        headers = []
        data = []
    
    # Append the extracted information to the DataFrame list
    df_list.append([fi_title, fi_price] + data)
    
    # Close the current tab and switch back to the previous tab
    try:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except:
        pass
#     driver.close()
#     driver.switch_to.window(driver.window_handles[0])

# Create DataFrame
df = pd.DataFrame(df_list, columns=['Title', 'Price'] + headers)

# Print DataFrame
print(df)

# Close the browser
# driver.quit()
df

ValueError: 2 columns passed, passed data had 22 columns

In [ ]:
# Initialize an empty DataFrame to store the results
df_list = []

# Iterate through XPATH to extract URLs and fetch information
for i in range(1, 74):
    # Extract the URL from the link element
    link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{i}]")))
    url = link_element.get_attribute("href")
    
    # Open the URL in a new tab
    driver.execute_script(f"window.open('{url}', '_blank')")
    
    # Switch to the newly opened tab
    driver.switch_to.window(driver.window_handles[-1])
    
    # Wait for the title and price elements to be present
    title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
    price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
    
    # Extract the title and price
    fi_title = title_element.text.strip()
    fi_price = price_element.text.strip()
    
    # Wait for the technical specifications section to be present
    tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))
    
    # Extract the outer HTML of the technical specifications section
    tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")
    
    # Parse the HTML content
    soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')
    
    # Find all rows in the table
    rows = soup.find('table').find_all('tr')
    
    # Extract column headers and data
    headers = []
    data = []
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(cols) == 2:  # Ensure there are two columns
            headers.append(cols[0].get_text(strip=True))
            data.append(cols[1].get_text(strip=True))
    
    # Add title and price at the beginning of the data list
    data.insert(0, fi_price)
    data.insert(0, fi_title)
    
    # Append the extracted information to the DataFrame list
    df_list.append([fi_title, fi_price] + data)
    
    # Close the current tab and switch back to the previous tab
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

# Create DataFrame
df = pd.DataFrame(df_list, columns=['Title', 'Price'] + headers)

# Print DataFrame
print(df)

# Close the browser
# driver.quit()
df

In [26]:
# Wait for the link element to be clickable
time.sleep(20)
link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[2]")))

# Extract the link
link = link_element.get_attribute("href")

# Print the link
print(link)

# Assign the extracted link to the URL variable
URL = link

# Fetch the webpage
driver.get(URL)

# Wait for the title and price elements to be present
title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))

# Extract the title and price
fi_title = title_element.text.strip()
fi_price = price_element.text.strip()

# Wait for the technical specifications section to be present
tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

# Extract the outer HTML of the technical specifications section
tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

# Parse the HTML content
soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

# Find all rows in the table
rows = soup.find('table').find_all('tr')

# Extract column headers and data
headers = []
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    if len(cols) == 2:  # Ensure there are two columns
        headers.append(cols[0].get_text(strip=True))
        data.append(cols[1].get_text(strip=True))

# Add title and price at the beginning of the data list
data.insert(0, fi_price)
data.insert(0, fi_title)

# Create DataFrame
df = pd.DataFrame([data], columns=['Title', 'Price'] + headers)

# Print DataFrame
print(df)

# Close the browser
# driver.quit()
# /html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a
# /html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[3]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a
# /html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a
# /html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[{}]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a

https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NzE3NzI2NDY2OTA0MTI1OjE3MTQ4ODY0MTU6c3BfYXRmOjIwMTUyMDE0NTk0NDk4OjowOjo&url=%2FLEGO-21330-Building-Adults-Pieces%2Fdp%2FB09GPP2BFJ%2Fref%3Dsr_1_2_sspa%3Fdib%3DeyJ2IjoiMSJ9.FpgwvNvVaFw5NgAQCdolOsbrbulhO0oYQarl9UQ9Y4hK5HUj9PnHneqpFQQVdvuDRBDvKInlQ0noJ7n1mVAUsCmGonESvOiZQhIb6DkpJ09VqQ6_Fv08LoIf_aDTyMHoiC7L_1s-gq3YMRORbNNdTzlx2EFC_f4ktLPs2_f7O9za7pu0FvbLDg2KSXbSGkWUGLvVRgExJyXu5C-AU283UQBXr6gHiZz7XCA3tJ6w01DE-ToepaWhMnQU3FOJPVgfn92TgTLWAdqqLVvD-5PKI3S-0Z_nQP_iQDjsh9_ybjE.6YbiBBD5V-LUh8L3nbz-7XDY7PXW657VxdDur0wIBns%26dib_tag%3Dse%26keywords%3Dlego%26qid%3D1714886415%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
                                               Title  \
0  LEGO Ideas Home Alone 21330 Building Kit; Grea...   

                             Price Language: Model Number  \
0  -50%\n₹17,499\nM.R.P.:\n₹34,999  ‎English       ‎21330   

  Number of Puzzle Pieces Assembly Required Batteries Required  \
0                  

In [27]:
df

,Title,Price,Language:,Model Number,Number of Puzzle Pieces,Assembly Required,Batteries Required,Batteries Included,Material Type(s),Colour,Product Dimensions,Item model number,Manufacturer recommended age,Manufacturer,Country of Origin,Item Weight
0,LEGO Ideas Home Alone 21330 Building Kit; Grea...,"-50%\n₹17,499\nM.R.P.:\n₹34,999",‎English,‎21330,‎3955,‎No,‎Yes,‎No,‎Plastic,‎Multicolor,‎17 x 48 x 58.2 cm; 2 kg,‎21330,‎4 years and up,"‎LEGO Toy Manufacturing NO. 2283, TONGXIANG RO...",‎China,‎2 kg


In [ ]:
# Wait for the link element to be clickable
link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a")))

# Extract the link
link = link_element.get_attribute("href")

# Print the link
print(link)

# Assign the extracted link to the URL variable
URL = link

# Fetch the webpage
driver.get(URL)

# Wait for the title and price elements to be present
title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
price_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))

# Extract the title and price
fi_title = title_element.text.strip()
fi_price = price_element.text.strip()

# Wait for the technical specifications section to be present
tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

# Extract the outer HTML of the technical specifications section
tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

# Parse the HTML content
soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

# Find all rows in the table
rows = soup.find('table').find_all('tr')

# Extract column headers and data
headers = []
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    if len(cols) == 2:  # Ensure there are two columns
        headers.append(cols[0].get_text(strip=True))
        data.append(cols[1].get_text(strip=True))

# Create DataFrame for technical specifications
tech_spec_df = pd.DataFrame([data], columns=headers)

# Create DataFrame for title and price
title_price_df = pd.DataFrame([[fi_title, fi_price]], columns=['Title', 'Price'])

# Concatenate both DataFrames
df = pd.concat([title_price_df, tech_spec_df], ignore_index=True)

# Print DataFrame
print(df)

# Close the browser
# driver.quit()


In [ ]:
# Wait for the link element to be clickable
link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a")))

# Extract the link
link = link_element.get_attribute("href")

# Print the link
print(link)

# Assign the extracted link to the URL variable
URL = link

# Fetch the webpage
driver.get(URL)

# Wait for the title and price elements to be present
title = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
price = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))

# Extract the title and price
fi_title = title.text.strip()
fi_price = price.text.strip()

# Wait for the technical specifications section to be present
tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

# Extract the outer HTML of the technical specifications section
tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

# Print the outer HTML
print("Technical Specifications Outer HTML:")
print(tech_spec_outer_html)


# Parse the HTML content
soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

# Find all rows in the table
rows = soup.find('table').find_all('tr')

# Extract column headers and data
headers = []
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    if len(cols) == 2:  # Ensure there are two columns
        headers.append(cols[0].get_text(strip=True))
        data.append(cols[1].get_text(strip=True))

# Create DataFrame
df = pd.DataFrame([data], columns=headers)

# Add title and price to DataFrame
df.loc[0, 'Title'] = fi_title
df.loc[0, 'Price'] = fi_price

# Print DataFrame
print(df)
df
# Close the browser
# driver.quit()


In [ ]:
# Wait for the link element to be clickable
link_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a")))

# Extract the link
link = link_element.get_attribute("href")

# Print the link
print(link)

# Assign the extracted link to the URL variable
URL = link

# Fetch the webpage
driver.get(URL)

title = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
price = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'corePriceDisplay_desktop_feature_div')))
fi_title = title.text
fi_price = price.text
# id="productTitle"
# id="corePriceDisplay_desktop_feature_div"

# Wait for the technical specifications section to be present
tech_spec_section = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productDetails_techSpec_section_1')))

# Extract the outer HTML of the technical specifications section
tech_spec_outer_html = tech_spec_section.get_attribute("outerHTML")

# Print the outer HTML
print("Technical Specifications Outer HTML:")
print(tech_spec_outer_html)


# Parse the HTML content
soup = BeautifulSoup(tech_spec_outer_html, 'html.parser')

# Find all rows in the table
rows = soup.find('table').find_all('tr')

# Extract column headers and data
headers = []
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    if len(cols) == 2:  # Ensure there are two columns
        headers.append(cols[0].get_text(strip=True))
        data.append(cols[1].get_text(strip=True))

# Create DataFrame
df = pd.DataFrame([data], columns=headers)

# Print DataFrame
print(df)
print(fi_title)
print(fi_price)
df

In [ ]:
# from bs4 import BeautifulSoup
# import pandas as pd

# html_content = """
# <table id="productDetails_techSpec_section_1" class="a-keyvalue prodDetTable" role="presentation">     
#     <tbody><tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Model Number </th>   
#         <td class="a-size-base prodDetAttrValue"> 21345 </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Number of Puzzle Pieces </th>   
#         <td class="a-size-base prodDetAttrValue"> 516 </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Assembly Required </th>   
#         <td class="a-size-base prodDetAttrValue"> Yes </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Batteries Required </th>   
#         <td class="a-size-base prodDetAttrValue"> No </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Batteries Included </th>   
#         <td class="a-size-base prodDetAttrValue"> No </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Material Type(s) </th>   
#         <td class="a-size-base prodDetAttrValue"> Plastic </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Colour </th>   
#         <td class="a-size-base prodDetAttrValue"> Multicolor </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Product Dimensions </th>   
#         <td class="a-size-base prodDetAttrValue"> 5.65 x 38.2 x 26.2 cm; 782 g </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Item model number </th>   
#         <td class="a-size-base prodDetAttrValue"> 21345 </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Manufacturer recommended age </th>   
#         <td class="a-size-base prodDetAttrValue"> 18 years and up </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Manufacturer </th>   
#         <td class="a-size-base prodDetAttrValue"> LEGO Toy Manufacturing NO. 2283, TONGXIANG ROAD, NANHU Dist. JIAXING CITY, ZHEJIANG, CHINA, 313014 </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Country of Origin </th>   
#         <td class="a-size-base prodDetAttrValue"> China </td>  
#     </tr>    
#     <tr>  
#         <th class="a-color-secondary a-size-base prodDetSectionEntry"> Item Weight </th>   
#         <td class="a-size-base prodDetAttrValue"> 782 g </td>  
#     </tr>
# </tbody></table>
# """

# # Parse the HTML content
# soup = BeautifulSoup(html_content, 'html.parser')

# # Find the table
# table = soup.find('table')

# # Extract column headers
# headers = [th.text.strip() for th in table.find_all('th')]

# # Extract data rows
# data = []
# for row in table.find_all('tr'):
#     data.append([td.text.strip() for td in row.find_all('td')])

# # Create DataFrame
# df = pd.DataFrame(data, columns=headers)

# # Print DataFrame
# print(df)


In [20]:
from bs4 import BeautifulSoup
import pandas as pd

html_content = """
<table id="productDetails_techSpec_section_1" class="a-keyvalue prodDetTable" role="presentation">     
    <tbody><tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Model Number </th>   
        <td class="a-size-base prodDetAttrValue"> 21345 </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Number of Puzzle Pieces </th>   
        <td class="a-size-base prodDetAttrValue"> 516 </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Assembly Required </th>   
        <td class="a-size-base prodDetAttrValue"> Yes </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Batteries Required </th>   
        <td class="a-size-base prodDetAttrValue"> No </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Batteries Included </th>   
        <td class="a-size-base prodDetAttrValue"> No </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Material Type(s) </th>   
        <td class="a-size-base prodDetAttrValue"> Plastic </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Colour </th>   
        <td class="a-size-base prodDetAttrValue"> Multicolor </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Product Dimensions </th>   
        <td class="a-size-base prodDetAttrValue"> 5.65 x 38.2 x 26.2 cm; 782 g </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Item model number </th>   
        <td class="a-size-base prodDetAttrValue"> 21345 </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Manufacturer recommended age </th>   
        <td class="a-size-base prodDetAttrValue"> 18 years and up </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Manufacturer </th>   
        <td class="a-size-base prodDetAttrValue"> LEGO Toy Manufacturing NO. 2283, TONGXIANG ROAD, NANHU Dist. JIAXING CITY, ZHEJIANG, CHINA, 313014 </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Country of Origin </th>   
        <td class="a-size-base prodDetAttrValue"> China </td>  
    </tr>    
    <tr>  
        <th class="a-color-secondary a-size-base prodDetSectionEntry"> Item Weight </th>   
        <td class="a-size-base prodDetAttrValue"> 782 g </td>  
    </tr>
</tbody></table>
"""

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all rows in the table
rows = soup.find('table').find_all('tr')

# Extract column headers and data
headers = []
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    if len(cols) == 2:  # Ensure there are two columns
        headers.append(cols[0].get_text(strip=True))
        data.append(cols[1].get_text(strip=True))

# Create DataFrame
df = pd.DataFrame([data], columns=headers)

# Print DataFrame
print(df)
df

  Model Number Number of Puzzle Pieces Assembly Required Batteries Required  \
0        21345                     516               Yes                 No   

  Batteries Included Material Type(s)      Colour  \
0                 No          Plastic  Multicolor   

             Product Dimensions Item model number  \
0  5.65 x 38.2 x 26.2 cm; 782 g             21345   

  Manufacturer recommended age  \
0              18 years and up   

                                        Manufacturer Country of Origin  \
0  LEGO Toy Manufacturing NO. 2283, TONGXIANG ROA...             China   

  Item Weight  
0       782 g  


,Model Number,Number of Puzzle Pieces,Assembly Required,Batteries Required,Batteries Included,Material Type(s),Colour,Product Dimensions,Item model number,Manufacturer recommended age,Manufacturer,Country of Origin,Item Weight
0,21345,516,Yes,No,No,Plastic,Multicolor,5.65 x 38.2 x 26.2 cm; 782 g,21345,18 years and up,"LEGO Toy Manufacturing NO. 2283, TONGXIANG ROA...",China,782 g


In [ ]:
URL = 'https://www.amazon.in/Lenovo-Display-Speakers-Octa-Core-Processor/dp/B0CY1S314G/ref=sr_1_1_sspa?dib=eyJ2IjoiMSJ9.Wi0jCS3ePpNscdPFu2huJuvPEHqr1lg_pMXfiVFFqJxkUZoLVtu54kaJ80hZD6J-oo853eJaVn7FXQcY8pvKx9y3AtC84MjF74D3pqWeVj4zTgy9sjLlhYsZZPIEd89CEVBPvlDf-4qgj4U4KkOzDDlmnr7Bm-YeONLtFwJ9A4whpl81Y-ne-wwF6Ph9NDg8ZDP3PfrxhFMMW_qV52xXeIwtmlxw4Ld7PZY88eW5rEo.NoQKb15tZoQG-uajGQ7TDv_ddBg4AXRK2qDXGb5LPZw&dib_tag=se&keywords=tablet&qid=1714756018&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

title = soup2.find(id='productTitle').get_text()

# price = soup2.find(id='priceblock_ourprice').get_text().strip()


print(title)
print(price)


In [11]:
Name_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[7]/div[3]/div[4]/div[1]/div/h1")))

# Extract the price text
Name = Name_element.text.strip()
print(Name)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007FF76F5F0EC4]
	(No symbol) [0x00007FF76F5F1052]
	(No symbol) [0x00007FF76F5E18A4]
	BaseThreadInitThunk [0x00007FFB4F017344+20]
	RtlUserThreadStart [0x00007FFB4F2E26B1+33]


In [8]:
# a 
price_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[4]/div/div/div/div/span/div/div/div[2]/div[3]/div[2]/div[1]")))

# Extract the price text
price = price_element.text.strip()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007FF76F5F0EC4]
	(No symbol) [0x00007FF76F5F1052]
	(No symbol) [0x00007FF76F5E18A4]
	BaseThreadInitThunk [0x00007FFB4F017344+20]
	RtlUserThreadStart [0x00007FFB4F2E26B1+33]


In [56]:
print(price)

₹8,999 M.R.P:
₹14,999 (40% off)


In [57]:
Name_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[5]/div/div/div/div/span/div/div/div[2]/div[1]/h2/a/span")))

# Extract the price text
Name = Name_element.text.strip()

In [58]:
print(Name)

LEGO Technic Kawasaki Ninja H2R Motorcycle Set 42170


In [21]:
# f
price_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/div/a[2]")))

# Extract the price text
Name = price_element.text.strip()

In [23]:
print(Name)

LEGO Speed Champions Valkyrie AMR Pro & Aston Martin Va...


In [27]:
price_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/div/a[3]/div[1]")))

# Extract the price text
price = price_element.text.strip()
print(price)

₹3,574₹5,49935% off


In [ ]:
# Wait for the title element to be clickable
title_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='productTitle']")))

# Extract the title text
title_outer_html = title_element.get_attribute("outerHTML")

# Print the outer HTML
print(title_outer_html)
title = title_element.text.strip()

# Wait for the price element to be clickable
price_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div/div/div/div/div/div[1]/div/div/div/div/div/div/div/div[6]/div/div/div/div/div/div[1]/div[2]/div/div/div/div/div/div/div[1]/div[2]/div")))

# Extract the price text
price = price_element.text.strip()

# Print the results
print(title)
print(price)


In [30]:
title1 = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//h1[@class='a-size-small a-color-secondary a-text-normal']")))
title1.text

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007FF76F5F0EC4]
	(No symbol) [0x00007FF76F5F1052]
	(No symbol) [0x00007FF76F5E18A4]
	BaseThreadInitThunk [0x00007FFB4F017344+20]
	RtlUserThreadStart [0x00007FFB4F2E26B1+33]


In [17]:
title_element = driver.find_element(By.XPATH, '//h1[@class="a-size-small a-color-secondary a-text-normal"]')

title = title_element.text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h1[@class="a-size-small a-color-secondary a-text-normal"]"}
  (Session info: chrome=124.0.6367.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007FF7

In [16]:
print(title)

Shop LEGO 


In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define the XPath expression
xpath_product_name = "/html/body/div[2]/div/div[5]/div[3]/div[4]/div[1]/div/h1/span"

# Set up the Selenium WebDriver (assuming you've downloaded and set up the appropriate driver for your browser)
# driver = webdriver.Chrome()  # Change to appropriate driver for your browser
driver.implicitly_wait(10)  # Implicit wait to handle dynamic loading of elements

try:
    # Load the webpage
#     driver.get("https://example.com")  # Replace with the URL of the webpage you want to scrape
    
    # Wait for the element to be visible
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_product_name)))

    # Find the element using XPath
    product_name_element = driver.find_element(By.XPATH, xpath_product_name)

    # Extract the text
    product_name = product_name_element.text

    print("Product Name:", product_name)

except Exception as e:
    print("An error occurred:", e)

# finally:
#     # Close the browser
#     driver.quit()


An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007FF76F5F0EC4]
	(No symbol) [0x00007FF76F5F1052]
	(No symbol) [0x00007FF76F5E18A4]
	BaseThreadInitThunk [0x00007FFB4F017344+20]
	RtlUserThreadStart [0x00007FFB4F2E26B1+33]



In [13]:
xpath_product_name = "(//span[@id='productTitle'])[1]"
company_name = driver.find_element(By.XPATH, xpath_product_name).text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[5]/div[3]/div[4]/div[1]/div/h1/span"}
  (Session info: chrome=124.0.6367.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007F

In [12]:
product_name_element = driver.find_element(By.XPATH, xpath_product_name)

# Extract the text
product_name = product_name_element.text

print("Product Name:", product_name)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[5]/div[3]/div[4]/div[1]/div/h1/span"}
  (Session info: chrome=124.0.6367.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007F

In [10]:
xpath_product_name = "/html/body/div[2]/div/div[5]/div[3]/div[4]/div[1]/div/h1/span"
xpath_Price = "//span[@class='a-size-small aok-offscreen']"
xpath_Price_discount = "(//span[@class='a-size-double-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'])[1]"
company_name = driver.find_element(By.XPATH, xpath_product_name).text
price = driver.find_element(By.XPATH, xpath_Price).text
discount = driver.find_element(By.XPATH, xpath_Price_discount).text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[5]/div[3]/div[4]/div[1]/div/h1/span"}
  (Session info: chrome=124.0.6367.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF76F671502+60802]
	(No symbol) [0x00007FF76F5EAC02]
	(No symbol) [0x00007FF76F4A7CE4]
	(No symbol) [0x00007FF76F4F6D4D]
	(No symbol) [0x00007FF76F4F6E1C]
	(No symbol) [0x00007FF76F53CE37]
	(No symbol) [0x00007FF76F51ABBF]
	(No symbol) [0x00007FF76F53A224]
	(No symbol) [0x00007FF76F51A923]
	(No symbol) [0x00007FF76F4E8FEC]
	(No symbol) [0x00007FF76F4E9C21]
	GetHandleVerifier [0x00007FF76F97411D+3217821]
	GetHandleVerifier [0x00007FF76F9B60B7+3488055]
	GetHandleVerifier [0x00007FF76F9AF03F+3459263]
	GetHandleVerifier [0x00007FF76F72B846+823494]
	(No symbol) [0x00007FF76F5F5F9F]
	(No symbol) [0x00007F

In [13]:
company_name

In [10]:
company_name_element
price
discount

In [37]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define the XPath
base_xpath_to_click = "//table[@id='productDetails_techSpec_section_1']"

try:
    # Wait for the element to be present on the page
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, base_xpath_to_click))
    )
    
    # Once the element is present, get its text
    price = element.get_attribute("outerHTML")
    print("Price:", price)

except Exception as e:
    print("Error:", e)


Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F1581502+60802]
	(No symbol) [0x00007FF7F14FAC02]
	(No symbol) [0x00007FF7F13B7CE4]
	(No symbol) [0x00007FF7F1406D4D]
	(No symbol) [0x00007FF7F1406E1C]
	(No symbol) [0x00007FF7F144CE37]
	(No symbol) [0x00007FF7F142ABBF]
	(No symbol) [0x00007FF7F144A224]
	(No symbol) [0x00007FF7F142A923]
	(No symbol) [0x00007FF7F13F8FEC]
	(No symbol) [0x00007FF7F13F9C21]
	GetHandleVerifier [0x00007FF7F188411D+3217821]
	GetHandleVerifier [0x00007FF7F18C60B7+3488055]
	GetHandleVerifier [0x00007FF7F18BF03F+3459263]
	GetHandleVerifier [0x00007FF7F163B846+823494]
	(No symbol) [0x00007FF7F1505F9F]
	(No symbol) [0x00007FF7F1500EC4]
	(No symbol) [0x00007FF7F1501052]
	(No symbol) [0x00007FF7F14F18A4]
	BaseThreadInitThunk [0x00007FF8428E7344+20]
	RtlUserThreadStart [0x00007FF8447A26B1+33]



In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import re


# Create lists to store data
data_list = []


    # Define the base XPath for the elements to click
base_xpath_to_click = "(//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])"
# Loop through elements
while True:

    for i in range(1, 29):  # Assuming you want to click on the first 5 elements
        # Construct the dynamic XPath
        xpath_to_click = base_xpath_to_click.format(i)
        try:
            # Wait for the element to be clickable
            element_to_click = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_to_click)))
            # Get text from the element to click
            jobtitle = element_to_click.text
            # Click on the element to open a new tab
            element_to_click.send_keys(Keys.CONTROL + Keys.RETURN)
            # Switch to the new tab
            driver.switch_to.window(driver.window_handles[1])
            # Define XPath for the element to extract text from in the new tab
            xpath_to_extract_text_new_tab = "/html/body/div[1]/main/div/div/div[2]/article/div/div/div[2]/div/app-root/div/vej-vacature-detail/div[4]/vej-vacature-content/div[1]/div[1]"
            # Wait for the element to be present on the new tab
            element_new_tab = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_to_extract_text_new_tab)))
            # Extract text from the element in the new tab
            decription = element_new_tab.text
            # Extract outer HTML from the element in the new tab
            decription1 = element_new_tab.get_attribute("outerHTML")
            # Extract additional information from new tab
            xpath_product_name = "(//h1[@id='title'])[1]"
            xpath_Price = "//span[@class='a-size-small aok-offscreen']"
            xpath_Price_discount = "(//span[@class='a-size-double-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'])[1]"
            company_name_element = driver.find_element(By.XPATH, xpath_product_name)
            companyname = company_name_element.text
            price = driver.find_element(By.XPATH, xpath_Price).text
            discount = driver.find_element(By.XPATH, xpath_Price_discount).text
            
            tag = driver.find_element(By.XPATH, xpath_tag).text
            # Extract company size and salary range if available
            # Append data to the list
            data_list.append([
            
            ])
            # Print or use the extracted link
    #            print("company_meta_outer_html:",company_meta_outer_html)  
    #            print("Link:", link_to_extract_text)
            print("Company Website:", company_website)
            print("Job:", jobtitle)
            print("Company Name:", companyname)
            print("Location:", location)
            print("Tag:", tag)
            print("Date of Posting:", dateofposting)
            print("Company Size:", company_size)
            print("Company Details:", company_details)
            print("Required Studies:", required_studies)
            print("Work Experience:", work_experience)
            print("Language Skills:", language_skills)
            print("Contract:", contract)
            print("Place of Employment:", place_of_employment)
            print("Compensation:", compensation)
            print("Description:", decription1)
            # Close the new tab
            driver.close()
            # Switch back to the original tab
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Error clicking on element {i}: {e}")
#          cookies
        css_selector_for_host1 = "#usercentrics-root"
        try:
            shadow_host_element = driver.find_element(By.CSS_SELECTOR, css_selector_for_host1)
            shadow_root = driver.execute_script('return arguments[0].shadowRoot', shadow_host_element)
            time.sleep(1)
            
            # Adjust the CSS selector as per your specific DOM structure within the shadow DOM
            target_element = shadow_root.find_element(By.CSS_SELECTOR, "div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > button:nth-child(2)").click()

        except:
            pass  # If the cookeis button is not found, continue with the loop

    try:
        # Click on the "Next Page" button
        css_selector_for_host1 = "vdab-pagination[pageamount='200']"
        
        # Replace sleep with WebDriverWait for better synchronization
        shadow_host_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector_for_host1))
        )
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', shadow_host_element)
        
        # Adjust the CSS selector as per your specific DOM structure within the shadow DOM
        target_element = shadow_root.find_element(By.CSS_SELECTOR, ".c-icon-text.c-pagination__arrow-link.-next")
        
        # Click on the target element
        target_element.click()
        
        
                # Add a delay of 2 seconds
        time.sleep(2)

    except:
        # If the "Next Page" button is not found, exit the loop
        break
# Close the original tab
# driver.quit()

# Create a DataFrame from the list
columns = ["job title", "company name", "location", "salary range", "tags", "date of post", "company size", "website", "Company Details", "Required Studies", "Work Experience", "Language Skills", "Contract", "Place of Employment", "Description"]
df = pd.DataFrame(data_list, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('ama.csv', index=False)

# Print the DataFrame
print(df)


Error clicking on element 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F1581502+60802]
	(No symbol) [0x00007FF7F14FAC02]
	(No symbol) [0x00007FF7F13B7CE4]
	(No symbol) [0x00007FF7F1406D4D]
	(No symbol) [0x00007FF7F1406E1C]
	(No symbol) [0x00007FF7F144CE37]
	(No symbol) [0x00007FF7F142ABBF]
	(No symbol) [0x00007FF7F144A224]
	(No symbol) [0x00007FF7F142A923]
	(No symbol) [0x00007FF7F13F8FEC]
	(No symbol) [0x00007FF7F13F9C21]
	GetHandleVerifier [0x00007FF7F188411D+3217821]
	GetHandleVerifier [0x00007FF7F18C60B7+3488055]
	GetHandleVerifier [0x00007FF7F18BF03F+3459263]
	GetHandleVerifier [0x00007FF7F163B846+823494]
	(No symbol) [0x00007FF7F1505F9F]
	(No symbol) [0x00007FF7F1500EC4]
	(No symbol) [0x00007FF7F1501052]
	(No symbol) [0x00007FF7F14F18A4]
	BaseThreadInitThunk [0x00007FF8428E7344+20]
	RtlUserThreadStart [0x00007FF8447A26B1+33]

Error clicking on element 2: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F1581502+60802]
	(No symbol) [0x00007FF7F14FAC02]
	(No symbol) [0

Empty DataFrame
Columns: [job title, company name, location, salary range, tags, date of post, company size, website, Company Details, Required Studies, Work Experience, Language Skills, Contract, Place of Employment, Description]
Index: []


In [14]:
import urllib.request

url = "https://www.amazon.in/s?k=tablet&ref=nb_sb_noss"
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36'}

request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)

html_bytes = response.read()
html_text = html_bytes.decode('utf-8', 'ignore')
print(html_text)


<!doctype html><html lang="en-in" class="a-no-js a-touch a-mobile" data-19ax5a9jf="mongoose"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta name="viewport" content="width=device-width, maximum-scale=1, minimum-scale=1, initial-scale=1, user-scalable=no, shrink-to-fit=no"/><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->

<script type='text/javascript'>var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta http-equiv='x-dns-prefetch-control' content='on'>
<link rel="dns-prefetch" href="https://images-eu.ssl-images-amazon.com" crossorigin>
<link rel="preconnect" href="https://images-eu.ssl-images-amazon.com" crossorigin>
<link rel="dns-prefetch" href="https://m.media-amazon.com" crossorigin>
<link rel="preconnect" href="https://m.media-amazon.com" crossorigin>
<link rel="dns-prefetch" href="https://completion.amazon.c

In [15]:
soup=BeautifulSoup(html_text,'html.parser')

In [16]:
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js a-touch a-mobile" data-19ax5a9jf="mongoose" lang="en-in">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta content="width=device-width, maximum-scale=1, minimum-scale=1, initial-scale=1, user-scalable=no, shrink-to-fit=no" name="viewport"/>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <!-- sp:feature:csm:head-open-part1 -->
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <!-- sp:end-feature:csm:head-open-part1 -->
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="preconnect"/>
  <link crossorigin="" href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link crossorigin="" href="https://m.media-amazon.com" rel="preconnec

In [17]:
soup.find_all('span',class_="a-offscreen")

[<span class="a-offscreen">₹26,999</span>,
 <span class="a-offscreen">₹39,999</span>,
 <span class="a-offscreen">₹39,900</span>,
 <span class="a-offscreen">₹44,900</span>,
 <span class="a-offscreen">₹20,999</span>,
 <span class="a-offscreen">₹27,999</span>,
 <span class="a-offscreen">₹36,500</span>,
 <span class="a-offscreen">₹46,999</span>,
 <span class="a-offscreen">₹21,999</span>,
 <span class="a-offscreen">₹2,399</span>,
 <span class="a-offscreen">₹4,499</span>,
 <span class="a-offscreen">₹26,999</span>,
 <span class="a-offscreen">₹39,999</span>,
 <span class="a-offscreen">₹11,299</span>,
 <span class="a-offscreen">₹26,000</span>,
 <span class="a-offscreen">₹14,999</span>,
 <span class="a-offscreen">₹28,999</span>,
 <span class="a-offscreen">₹9,500</span>,
 <span class="a-offscreen">₹21,000</span>,
 <span class="a-offscreen">₹30,900</span>,
 <span class="a-offscreen">₹10,999</span>,
 <span class="a-offscreen">₹20,999</span>,
 <span class="a-offscreen">₹12,998</span>,
 <span class="

In [18]:
for i in soup.find_all('div',class_="a-section"):
  print(i.text.strip())


and Up







Up to 6.9 Inch






7.0 - 7.9 Inch






8.0 - 8.9 Inch






9 Inch & Above






Under ₹1,000






₹1,000 - ₹5,000






₹5,000 - ₹10,000






₹10,000 - ₹20,000






Over ₹20,000






Bluetooth






Cellular






Wi-Fi






17 GB & Above






Samsung








Filters
and Up







Up to 6.9 Inch






7.0 - 7.9 Inch






8.0 - 8.9 Inch






9 Inch & Above






Under ₹1,000






₹1,000 - ₹5,000






₹5,000 - ₹10,000






₹10,000 - ₹20,000






Over ₹20,000






Bluetooth






Cellular






Wi-Fi






17 GB & Above






Samsung








Filters

and Up
and Up





Up to 6.9 Inch
7.0 - 7.9 Inch
8.0 - 8.9 Inch
9 Inch & Above
Under ₹1,000
₹1,000 - ₹5,000
₹5,000 - ₹10,000
₹10,000 - ₹20,000
Over ₹20,000
Bluetooth
Cellular
Wi-Fi
17 GB & Above
Samsung
Filters
Filters
Xiaomi Pad 6 - Do It BetterSponsored
Shop XiaomiShop now

Filter by tablet screen size








Up to 6.9 Inch






7.0 - 7.9 Inch






8.0 - 8.9 Inch






9 Inch & Above







Apply
Filter by

In [19]:
h3 = []

for i in soup.find_all('span',class_="a-price-whole"):
    h3.append(i.text.strip())

df = pd.DataFrame(h3, columns=['Price'])

print(df)


     Price
0   26,999
1   39,900
2   20,999
3   36,500
4   21,999
5    2,399
6   26,999
7   11,299
8   14,999
9    9,500
10  30,900
11  10,999
12  12,998
13  11,499
14   8,999
15   9,500
16  19,999
17  12,949
18  20,999
19   7,999
20  14,999
21  30,900
22  16,998
23   9,999
24     815
25  15,390
26  14,999


In [20]:
df

,Price
0,"26,999"
1,"39,900"
2,"20,999"
3,"36,500"
4,"21,999"
5,"2,399"
6,"26,999"
7,"11,299"
8,"14,999"
9,"9,500"
